In [1]:
import sys
sys.path.append('../../')
import django,os,glob
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

In [2]:
import tensorflow as tf

In [3]:
from dvalib import detector

In [4]:
class_names = {1: {'id': 1, 'name': u'person'},
               2: {'id': 2, 'name': u'bicycle'},
               3: {'id': 3, 'name': u'car'},
               4: {'id': 4, 'name': u'motorcycle'},
               5: {'id': 5, 'name': u'airplane'},
               6: {'id': 6, 'name': u'bus'},
               7: {'id': 7, 'name': u'train'},
               8: {'id': 8, 'name': u'truck'},
               9: {'id': 9, 'name': u'boat'}, 
               10: {'id': 10, 'name': u'traffic light'}, 
               11: {'id': 11, 'name': u'fire hydrant'},
               13: {'id': 13, 'name': u'stop sign'}, 
               14: {'id': 14, 'name': u'parking meter'},
               15: {'id': 15, 'name': u'bench'}, 
               16: {'id': 16, 'name': u'bird'}, 
               17: {'id': 17, 'name': u'cat'}, 
               18: {'id': 18, 'name': u'dog'}, 
               19: {'id': 19, 'name': u'horse'}, 
               20: {'id': 20, 'name': u'sheep'}, 
               21: {'id': 21, 'name': u'cow'}, 
               22: {'id': 22, 'name': u'elephant'}, 
               23: {'id': 23, 'name': u'bear'}, 
               24: {'id': 24, 'name': u'zebra'},
               25: {'id': 25, 'name': u'giraffe'},
               27: {'id': 27, 'name': u'backpack'},
               28: {'id': 28, 'name': u'umbrella'},
               31: {'id': 31, 'name': u'handbag'},
               32: {'id': 32, 'name': u'tie'},
               33: {'id': 33, 'name': u'suitcase'},
               34: {'id': 34, 'name': u'frisbee'}, 
               35: {'id': 35, 'name': u'skis'},
               36: {'id': 36, 'name': u'snowboard'}, 
               37: {'id': 37, 'name': u'sports ball'},
               38: {'id': 38, 'name': u'kite'},
               39: {'id': 39, 'name': u'baseball bat'},
               40: {'id': 40, 'name': u'baseball glove'},
               41: {'id': 41, 'name': u'skateboard'},
               42: {'id': 42, 'name': u'surfboard'},
               43: {'id': 43, 'name': u'tennis racket'},
               44: {'id': 44, 'name': u'bottle'}, 
               46: {'id': 46, 'name': u'wine glass'}, 
               47: {'id': 47, 'name': u'cup'}, 
               48: {'id': 48, 'name': u'fork'}, 
               49: {'id': 49, 'name': u'knife'},
               50: {'id': 50, 'name': u'spoon'}, 
               51: {'id': 51, 'name': u'bowl'}, 
               52: {'id': 52, 'name': u'banana'}, 
               53: {'id': 53, 'name': u'apple'},
               54: {'id': 54, 'name': u'sandwich'},
               55: {'id': 55, 'name': u'orange'}, 
               56: {'id': 56, 'name': u'broccoli'},
               57: {'id': 57, 'name': u'carrot'}, 
               58: {'id': 58, 'name': u'hot dog'}, 
               59: {'id': 59, 'name': u'pizza'},
               60: {'id': 60, 'name': u'donut'}, 
               61: {'id': 61, 'name': u'cake'},
               62: {'id': 62, 'name': u'chair'}, 
               63: {'id': 63, 'name': u'couch'}, 
               64: {'id': 64, 'name': u'potted plant'}, 
               65: {'id': 65, 'name': u'bed'}, 
               67: {'id': 67, 'name': u'dining table'},
               70: {'id': 70, 'name': u'toilet'},
               72: {'id': 72, 'name': u'tv'}, 
               73: {'id': 73, 'name': u'laptop'}, 
               74: {'id': 74, 'name': u'mouse'},
               75: {'id': 75, 'name': u'remote'}, 
               76: {'id': 76, 'name': u'keyboard'}, 
               77: {'id': 77, 'name': u'cell phone'},
               78: {'id': 78, 'name': u'microwave'}, 
               79: {'id': 79, 'name': u'oven'}, 
               80: {'id': 80, 'name': u'toaster'},
               81: {'id': 81, 'name': u'sink'}, 
               82: {'id': 82, 'name': u'refrigerator'}, 
               84: {'id': 84, 'name': u'book'},
               85: {'id': 85, 'name': u'clock'},
               86: {'id': 86, 'name': u'vase'}, 
               87: {'id': 87, 'name': u'scissors'},
               88: {'id': 88, 'name': u'teddy bear'},
               89: {'id': 89, 'name': u'hair drier'},
               90: {'id': 90, 'name': u'toothbrush'}}

In [5]:
class_index_to_string = {k:v['name'] for k,v in class_names.iteritems()} 

In [12]:
print class_index_to_string

{1: u'person', 2: u'bicycle', 3: u'car', 4: u'motorcycle', 5: u'airplane', 6: u'bus', 7: u'train', 8: u'truck', 9: u'boat', 10: u'traffic light', 11: u'fire hydrant', 13: u'stop sign', 14: u'parking meter', 15: u'bench', 16: u'bird', 17: u'cat', 18: u'dog', 19: u'horse', 20: u'sheep', 21: u'cow', 22: u'elephant', 23: u'bear', 24: u'zebra', 25: u'giraffe', 27: u'backpack', 28: u'umbrella', 31: u'handbag', 32: u'tie', 33: u'suitcase', 34: u'frisbee', 35: u'skis', 36: u'snowboard', 37: u'sports ball', 38: u'kite', 39: u'baseball bat', 40: u'baseball glove', 41: u'skateboard', 42: u'surfboard', 43: u'tennis racket', 44: u'bottle', 46: u'wine glass', 47: u'cup', 48: u'fork', 49: u'knife', 50: u'spoon', 51: u'bowl', 52: u'banana', 53: u'apple', 54: u'sandwich', 55: u'orange', 56: u'broccoli', 57: u'carrot', 58: u'hot dog', 59: u'pizza', 60: u'donut', 61: u'cake', 62: u'chair', 63: u'couch', 64: u'potted plant', 65: u'bed', 67: u'dining table', 70: u'toilet', 72: u'tv', 73: u'laptop', 74: u'm

In [6]:
model_path = "/Users/aub3/Dropbox/DeepVideoAnalytics/dvalib/object_detection/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb"
d = detector.TFDetector(model_path=model_path,class_index_to_string=class_index_to_string)

In [7]:
d.load()

In [10]:
detections = d.detect("/Users/aub3/Dropbox/DeepVideoAnalytics/notebooks/images/person.jpg")

In [11]:
for d in detections:
    print d

{'right': 275, 'bottom': 381, 'top': 103, 'object_name': u'person', 'score': 0.98659921, 'left': 193}
{'right': 600, 'bottom': 340, 'top': 135, 'object_name': u'cow', 'score': 0.82458395, 'left': 408}
{'right': 202, 'bottom': 347, 'top': 266, 'object_name': u'dog', 'score': 0.70820665, 'left': 72}
{'right': 601, 'bottom': 347, 'top': 138, 'object_name': u'sheep', 'score': 0.37876359, 'left': 403}
{'right': 604, 'bottom': 343, 'top': 136, 'object_name': u'dog', 'score': 0.1835705, 'left': 404}


### Trying with queue

In [ ]:
input_path = "/Users/aub3/media/1/frames/*.jpg"
model_path = "/Users/aub3/Dropbox/DeepVideoAnalytics/dvalib/object_detection/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb"
detection_graph = tf.Graph()
with detection_graph.as_default():
    filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once(input_path))
    image_reader = tf.WholeFileReader()
    imname, image_file = image_reader.read(filename_queue)
    image = tf.expand_dims(tf.image.decode_image(image_file),0)
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(model_path, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='',input_map={"image_tensor:0": image})
    sess = tf.InteractiveSession(graph=detection_graph)
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
scores = detection_graph.get_tensor_by_name('detection_scores:0')
classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
imname